In [1]:
import os

data_path = "data/natural"

# with open(os.path.join(data_path, "sharegpt_7b_400.txt"), "r") as f:
#     text = f.read()
#     transcripts = text.split("[[ID]]")[1:]
#     print(len(transcripts))

# transcripts_range = transcripts[200:224]
# text_range = "[[ID]]" + "[[ID]]".join(transcripts_range)
# with open(os.path.join(data_path, "Ann200-224_sharegpt_7b.txt"), "w") as f:
#     f.write(text_range)
# print(len(transcripts_range))

In [11]:
import re
import os

TAGS = "SUBJ, AMBIG, COM, CODE, Q, STY, FIC, LANG, NATT, NATF, FLAG, PII, STALE".split(", ")
LABELS = "N/A, True, Maybe True, Uncertain, Maybe False, False".split(", ")
annotated_path = os.path.join(data_path, "annotated")
with open(os.path.join(annotated_path, "alex50-64_sharegpt_7b.txt"), "r") as f:
    text = "\n\n" + f.read()

transcripts = text.split("\n\n[[ID]] ")[1:]
parent_ids = []
assistant_texts = []
all_labels = []
all_tags = []

for i, trans in enumerate(transcripts):
    parent_id = trans[:trans.index("\n")]
    asst_text = trans[trans.rindex("\n\nASSISTANT: ") + len("\n\nASSISTANT: "):]
    parent_ids.append(parent_id)
    assistant_texts.append(asst_text)

    # should match "[[1|", "[[ 1 |", "[[ 2|", "[[12 |"
    tag_pattern = r"\[\[\s*\d+\s*\|"

    labels = []
    tags = []
    match_iter = re.finditer(tag_pattern, asst_text)
    matches = []
    match = None
    for match_num, match in enumerate(match_iter):
        # find next instance of "]]" after match
        end = asst_text.find("]]", match.end())
        if end == -1:
            print(f"WARNING: no end bracket found for match {match}")
            continue
        annotation = asst_text[match.end():end].strip()
        if "#" in annotation:
            first_hash = annotation.index("#")
            label, tag_text = annotation[:first_hash], annotation[first_hash + 1:]
        else:
            label = annotation
            tag_text = ""
        label = label.strip()
        if label not in LABELS:
            print(f"WARNING: unknown label {label}")
        labels.append(label)
        for tag in TAGS:
            if tag in tag_text:
                tags.append(tag)
                break
        
        matches.append(match.group())
    
    all_labels.append(labels)
    all_tags.append(tags)

    if match:
        last_match = match.group()
        last_num = int(last_match[2:-1].strip())
        if last_num != match_num + 1:
            print(f"WARNING: last number does not match number of matches in transcript {i}")
            # print(matches)
            # print(asst_text)


In [12]:
import numpy as np

l = np.concatenate(all_labels)
t = np.concatenate(all_tags)

In [13]:
sum("false" in lab.lower() for lab in l), len(l)

(22, 358)

In [10]:
len(t)

255